In [479]:
with open('16a.txt') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

In [480]:
def hex_to_bin(hex_in: str) -> str:
    # d = bin(int(hex_in, 16))[2:]
    # while len(d) < 4 * len(hex_in):
    #     d = '0' + d
    # return d
    return format(int(hex_in.strip(), 16), f"0{len(hex_in.strip()) * 4}b")

In [481]:
binary = hex_to_bin(lines[0])

### Part 1

In [482]:
# binary = hex_to_bin("C0015000016115A2E0802F182340")

In [483]:
def literal_decode(super_packet: str):
    # return both the version number and the last index of the super_packet that contains the literal packet
    # print("literal packet: " + super_packet)
    version = int(super_packet[:3], 2)
    done = False
    i = 6
    value = ""
    while not done:
        value = value + super_packet[i+1:i+5]
        # print(super_packet[i:i+5])
        if super_packet[i] == '0':
            done = True
            # print('done literal.')
        i += 5
    return (i-1, version)


In [484]:
def operator_decode(super_packet: str):
    # print("operator!")
    # print("operator packet: " + super_packet)
    # super packet is a binary string that represents all of the remaining binary
    # where literally all of it technically could be this operator packet
    version = int(super_packet[:3],2)
    sub_version_sum = 0
    packet_id = int(super_packet[3:6],2)
    if packet_id == 4: print("treated literal as operator oops :(")
    length_type = super_packet[6]
    if length_type == '1':
        # print("ltype_id = 1")
        num_sub_packs = int(super_packet[7:18],2)
        counter = 0
        init_index = 18
        index = init_index
        while (counter < num_sub_packs):
            sub_packet_type = int(super_packet[index+3:index+6],2)
            if sub_packet_type != 4:
                # operator
                new_index, new_version = operator_decode(super_packet[index:])
                sub_version_sum += new_version
                index += new_index
                # print("index: " + str(index))
            else:
                new_index, new_version = literal_decode(super_packet[index:])
                sub_version_sum += new_version
                index += 1 + new_index
            counter += 1
    else:
        total_length = int(super_packet[7:22],2)
        length_counter = 0
        init_index = 22
        index = init_index
        while (length_counter < total_length):
            sub_packet_type = int(super_packet[index+3:index+6],2)
            # print(sub_packet_type)
            if sub_packet_type != 4:
                # operator
                new_index, new_version = operator_decode(super_packet[index:])
                sub_version_sum += new_version
                length = new_index
                index += length
            else:
                new_index, new_version = literal_decode(super_packet[index:])
                sub_version_sum += new_version
                # print(new_index)
                length = new_index + 1
                index += length
            length_counter += length
            # print("CURR LENGTH: " + str(length) + " LENGTH COUNTER: " + str(length_counter))
            # print(index)
    return (index, sub_version_sum + version)


In [485]:
print(operator_decode(binary))

(5337, 893)


In [486]:
# print(binary) #version outside op = 3
# print(binary[18:]) 
# sub = binary[18:] #inside the outside op\
# # print(sub[:3]) # version of op 1  = 0
# # print(sub[7:22]) # 22
# lit1 = sub[22:]
# print("lit1: " + lit1) # lit1 version = 0
# print(lit1[3:6]) # lit1 type = 4 (good)
# print(lit1[6:11]) # block 1
# lit2 = lit1[11:] 
# print("lit2: " + lit2) # lit2 version = 5
# print(lit2[6:11]) # end of operator 1
# op2 = lit2[11:]
# print("op2: " + op2) # op2 version = 1, type = 0
# print(op2[:7]) # op2 len type id = 1 -> next 11 bits are whatever
# print()


### Part 2

In [487]:
def improved_literal_decode(super_packet: str):
    # returns the index and the value
    version = int(super_packet[:3], 2)
    done = False
    i = 6
    value = ""
    while not done:
        value = value + super_packet[i+1:i+5]
        # print(value)
        # print(super_packet[i:i+5])
        if super_packet[i] == '0':
            done = True
            # print('done literal.')
        i += 5
    dec_val = int(value, 2)
    if dec_val < 0: print("wtf?!")
    return (i-1, dec_val)


In [488]:
def improved_operator_decode(super_packet: str):
    # 0 -> SUM
    # 1 -> PRODUCT
    # 2 -> MINIMUM
    # 3 -> MAXIMUM
    # 5 -> GREATER THAN
    # 6 -> LESS THAN
    # 7 -> EQUAL TO
    packet_id = int(super_packet[3:6],2)
    if packet_id == 4: print("treated literal as operator oops :(")
    length_type = super_packet[6]
    values  = []
    if length_type == '1':
        # print("ltype_id = 1")
        num_sub_packs = int(super_packet[7:18],2)
        counter = 0
        init_index = 18
        index = init_index
        while (counter < num_sub_packs):
            sub_packet_type = int(super_packet[index+3:index+6],2)
            if sub_packet_type != 4:
                # operator
                new_index, value = improved_operator_decode(super_packet[index:])
                index += new_index
                values.append(value)
                # print("index: " + str(index))
            else:
                new_index, value = improved_literal_decode(super_packet[index:])
                index += 1 + new_index
                values.append(value)
            counter += 1
    else:
        total_length = int(super_packet[7:22],2)
        length_counter = 0
        init_index = 22
        index = init_index
        while (length_counter < total_length):
            sub_packet_type = int(super_packet[index+3:index+6],2)
            # print(sub_packet_type)
            if sub_packet_type != 4:
                # operator
                new_index, value = improved_operator_decode(super_packet[index:])
                length = new_index
                index += length
                values.append(value)
            else:
                new_index, value = improved_literal_decode(super_packet[index:])
                # print(new_index)
                length = new_index + 1
                index += length
                values.append(value)
            length_counter += length
            # print("CURR LENGTH: " + str(length) + " LENGTH COUNTER: " + str(length_counter))
            # print(index)
    if packet_id == 0:
        final_val = sum(values)
    elif packet_id == 1:
        final_val = 1
        for val in values:
            final_val *= val
    elif packet_id == 2:
        final_val = min(values)
    elif packet_id == 3:
        final_val = max(values)
    elif packet_id == 5:
        if values[0] > values[1]: final_val = 1
        else: final_val = 0
    elif packet_id == 6:
        if values[0] < values[1]: final_val = 1
        else: final_val = 0
    elif packet_id == 7:
        if values[0] == values[1]: final_val = 1
        else: final_val = 0
    return (index,final_val)


In [489]:
binary = hex_to_bin(lines[0])

In [491]:
improved_operator_decode(binary)

(5337, 4358595186090)